In [35]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [36]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from tkinter import ttk

import random
import json 
import nltk

In [37]:
#Menu Screen:
Menu = tk.Tk()
Menu.title("Bootleg Wordle Menu")
Menu.geometry("600x800")

Welcome_Message = tk.Label(Menu, text = "Welcome to the main menu of Bootleg Wordle! \n" 
                                       "Here, you can select a difficulty, and learn how to play.", 
                           font=('Times New Roman',18)
                          )
Welcome_Message.pack()

How_to_play = tk.Label(Menu, text="How to play:")
How_to_play.pack(pady=10)

Rules_Message = tk.Label(Menu, text="Wordle is a word guessing game.\n"
                                   "You have 6 guesses to match a randomly chosen word!\n"
                                   "Each guess must be 5 letters long.\n"
                                   "The color of the tiles will change to show how close\n"
                                   "your guess was to the word.",
                        font=('Times New Roman', 15)
                        )
Rules_Message.pack(pady=10)  


# Define the difficulty label
Rules_Message.pack()
difficulty_var = tk.StringVar()

label_difficulty = tk.Label(Menu, text="Please select a Difficulty:", font=('Arial', 18))
label_difficulty.pack(pady=10)

style = ttk.Style()
style.configure("TRadiobutton", indicatorsize=0)

radio_easy = ttk.Radiobutton(Menu, text="Easy", variable=difficulty_var, value="Easy", style="TRadiobutton", padding=10)
radio_easy.pack()

radio_medium = ttk.Radiobutton(Menu, text="Medium", variable=difficulty_var, value="Medium", style="TRadiobutton", padding=10)
radio_medium.pack()

radio_hard = ttk.Radiobutton(Menu, text="Hard", variable=difficulty_var, value="Hard", style="TRadiobutton", padding=10)
radio_hard.pack()



#Timer Initialization
timer_label = None
countdown_seconds = 60 #set countdown 60 second
countdown_id = None
actualWord = ""

# Function to start the game when given difficulty level selection:
def start_game(difficulty):
    #reference - https://www.reddit.com/r/learnpython/comments/11w3cad/created_my_first_tkinter_project_countdown_timer/
    global countdown_seconds, countdown_id, timer_label, root
    if difficulty == "Hard":
        try:
            # Check if the root window exists
            # reference -https://stackoverflow.com/questions/29076086/see-if-tkinter-widget-exists-not-using-winfo-exists
            if not root.winfo_exists():
                return  # Exit if the root window has been destroyed

            if timer_label is None:
                countdown_seconds = 60
                timer_label = tk.Label(root, text=f"Time Left: {countdown_seconds} seconds", font=('Arial', 18))
                timer_label.pack()
                countdown_id = root.after(1000, countdown)
        except tk.TclError:
            # Handle TclError if the root window has been destroyed
            pass

# Function to read selected difficulty and start the game with that difficulty
def select_difficulty(difficulty_var):
    difficulty = difficulty_var.get()
    if difficulty:
        start_game(difficulty)
        # Destroy the menu window when the game starts
        Menu.destroy()  
    else:
        # reference -https://docs.python.org/3/library/tkinter.messagebox.html
        messagebox.showwarning("Difficulty Selection", "Please select a difficulty level.")

        
first_guess_made = False
 
    
# Countdown function
# reference - https://stackoverflow.com/questions/72648183/how-to-stop-tkinter-timer-function-when-i-press-button-one-more-times
def countdown():
    global countdown_seconds, countdown_id, timer_label, gylist
    
    # reference - https://colinch4.github.io/2023-09-06/13-10-15-355782/
    if countdown_seconds > 0:
        countdown_seconds -= 1
        if timer_label:
            timer_label.config(text=f"Time Left: {countdown_seconds} seconds")

        # Update button colors based on the guessed letters
        guess = textbox.get("1.0", "end-1c").upper().strip()

        # Initialize gylist if not already initialized
        # reference - https://stackoverflow.com/questions/68585252/how-to-check-if-a-global-variable-exists-and-if-not-then-define-it-as-global
        if 'gylist' not in globals():
            gylist = ['#000000'] * 5

        # Update gylist based on the current guess
        gylist = update_gylist(guess, gylist)

        # Update button colors
        update_button_colors(guess, gylist)

        # Call the countdown function again after 1000 milliseconds (1 second)
        # reference - https://stackoverflow.com/questions/36402134/tkinter-root-after-cancel
        if countdown_id:
            # cancel the previous scheduled call
            root.after_cancel(countdown_id)  
        countdown_id = root.after(1000, countdown)
     
    # reference - https://stackoverflow.com/questions/53580507/disable-enable-button-in-tkinter
    if countdown_seconds <= 0:
        lose_Win_Length_label.config(text=f"You Lose! The word was {actualWord}")
        btnSubmit.config(state=DISABLED)
        return

def update_gylist(guess, gylist):
    # Update gylist based on the current guess
    # reference  -https://stackoverflow.com/questions/19973363/how-can-i-get-a-tkinter-widget-attribute
    for i, letter in enumerate(guess):
        if i < len(gylist):
            position = Alphabet.index(letter)
            gylist[i] = buttons[position].cget("fg")
    return gylist

def update_button_colors(guess, gylist):
    # Update button colors
    for i, letter in enumerate(guess):
        if i < len(gylist):
            position = Alphabet.index(letter)
            current_color = buttons[position].cget("fg")
            if current_color != gylist[i]:
                buttons[position].config(fg=gylist[i])

Start_Game = tk.Button(Menu, text="Start Game", command=lambda: select_difficulty(difficulty_var))
Start_Game.pack()

Menu.mainloop()



#End of Menu -- Start of Game screen:


#Import two dictionary

# Get json words
# Reference- How to Parse JSON
#https://www.freecodecamp.org/news/loading-a-json-file-in-python-how-to-read-and-parse-json/

#you should change this path of your computer where downloaded json file
json_file_path = r'C:\Users\Dell\Downloads\words.json'
with open(json_file_path, 'r') as json_file:
    json_data = json.load(json_file)

# Download NLTK words
#Reference - NLTK website
#https://www.nltk.org/data.html
nltk.download('words')

# Get NLTK words
nltk_words = set([word.upper() for word in nltk.corpus.words.words() if len(word) == 5])

# Combine both dictionaries
combined_words = list(set(json_data + list(nltk_words)))

# Choose random word function
# Reference - use random
#https://stackoverflow.com/questions/1456617/return-a-random-word-from-a-word-list-in-python
def choose_random_word():
    random_word = random.choice(combined_words)
    return random_word.upper() 

# Chooses Word
actualWord = choose_random_word()

turn=1

       
#Creates GUI screen with title and dimension:
root = tk.Tk()
root.title("Bootleg Wordle")
root.geometry("600x800")


#Creates textbox in GUI to type guess in
textbox = tk.Text(root, height=2, font=('Arial',18))
textbox.pack(padx=20,pady=5)


#Label asking for guess:
label = tk.Label(root, text='Please Enter a Guess!', font=('Arial',18))
label.pack()


#Function that will be used inside to make the following keyboard work:
def insert_text(letter):
    current_text = textbox.get("1.0", "end-1c")
    new_text = current_text + letter
    textbox.delete("1.0", "end")
    textbox.insert("1.0", new_text)


#Creates button frame to hold keyboard buttons:
buttonframe = tk.Frame(root)
buttonframe.columnconfigure(0, weight=1)
buttonframe.columnconfigure(1, weight=1)
buttonframe.columnconfigure(2, weight=1)

Alphabet = "QWERTYUIOPASDFGHJKLZXCVBNM"
buttons = []
#Places each button in qwerty order in button frame.
#Each button is assigned the "insert_text" command to actually write the letter when clicked:
for index, letter in enumerate(Alphabet):
    btnltr = tk.Button(buttonframe, text=letter, font=('Arial', 18), command=lambda l=letter: insert_text(l))
    if(index <=9):
        btnltr.grid(row=0, column=index, sticky=tk.W + tk.E)
        
    elif(index >=10 and index <=18):
        btnltr.grid(row=1, column=index-10, sticky=tk.W + tk.E)
        
    else:
        btnltr.grid(row=2, column=index-19, sticky=tk.W + tk.E)
    buttons.append(btnltr)
       
buttonframe.pack()

#Creates function for Backspace button:
def Backspace():
    textbox.delete("end-2c", "end")

btnDelete = tk.Button(buttonframe, text='Delete', font=('Arial',18),command=Backspace)
btnDelete.grid(row=2,column=7,columnspan=2,sticky=tk.W+tk.E)
buttonframe.pack()
#Creates function for clearing the test box after a guess is submitted:
def ClearText():
    textbox.delete("1.0", "end")


# Initialize global turn variable
global turn
turn=1

def Actual_Game():
    global turn, actualWord, countdown_seconds, timer_label, countdown_id,GuessLabels

    # Get the difficulty level after it has been selected
    difficulty = difficulty_var.get()

    # Create or update the timer_label based on the difficulty
    if difficulty == "Hard" and timer_label is None:
        timer_label = tk.Label(root, text=f"Time Left: {countdown_seconds} seconds", font=('Arial', 18))
        timer_label.pack()

        # Start the countdown when difficulty is set to "Hard"
        countdown_id = root.after(1000, countdown)
        
    if timer_label is not None:
        countdown()

    #Function to check if all entries in a list are all equal
    def check(list):
        return all(i == list[0] for i in list)

    Black = '#000000'
    Yellow = '#c8b653'
    Green = '#6ca965'
    
    # Auto-guess algorithm
    # if auto:
    #     if turn == 1:
    #         guess = Choose_Random_Word()
    #     else:
    #         feasible_words=data
    #         for word in data:
    #             for ind, color in enumerate(gylist):
    #                 if (color==Green and guess[ind]!=word[ind]):
    #                     feasible_words.remove(word)
    #                 if (color==Yellow and (guess[ind] not in word)):
    #                     feasible_words.remove(word)
            
            # guess = random.choice(feasible_words).upper()
            # lose_Win_Length_label.config(text=f"Auto-guess: {guess}")
    
    #Pulls the user's guess from the textbox; Makes it capitalalized and removes blank spaces:
    guess = textbox.get("1.0", "end-1c").upper().strip()
    Black = '#000000'
    Yellow = '#c8b653'
    Green = '#6ca965'
    
    #Colors = {'Black' : '#000000', 'Yellow':'#c8b653', Green:'#6ca965'}
    gylist = [Black,Black,Black,Black,Black]

    #If guess length isn't 5, don't run the game and tell the user:
    if len(guess)!=5:
        lose_Win_Length_label.config(text="Only 5 Letter Words!")
        
    #If 5 letter word is entered, play the game!:
    else:
        lose_Win_Length_label.config(text="")
        #Find all Green letters:
        if turn <=6:
            for gcNum, guessChar  in enumerate(guess):
                for acNum, acChar in enumerate(actualWord):
                    if (guessChar == acChar and gcNum == acNum):
                        gylist[gcNum] = Green
            
            #Find all Yellow letters
            for acNum, acChar in enumerate(actualWord):
                if (gylist[acNum] != Green):
                     for gcNum, guessChar  in enumerate(guess):
                        # Make sure character has not already been colored
                          if (guessChar == acChar and gylist[gcNum] == Black):
                                gylist[gcNum] = Yellow
                    
            #Once one yellow character has been found corresponding to a specific actual character, break inner-loop
                          break
            for i, letter in enumerate(guess):
                position = Alphabet.index(letter)
                current_color = buttons[position].cget("fg")
                if current_color != Green and current_color != Yellow and gylist[i]!=Black:
                    buttons[position].config(fg = gylist[i])
                elif current_color != Green and current_color != Yellow and gylist[i]==Black:
                    buttons[position].config(fg = '#808080')     
                elif current_color == Yellow and gylist[i]==Green:
                    buttons[position].config(fg = gylist[i])   
            
            #Makes Frame to layout the Wordle guesses vertically:
            GuessLabels = tk.Frame(root)
            GuessLabels.columnconfigure(3, weight=1)
            GuessLabels.columnconfigure(4, weight=1)
            GuessLabels.columnconfigure(5, weight=1)
        
            #Creates Label for each letter in guess, colored accordingly
            k=0
            for k in range(0,5):
                GuessLetter = tk.Label(GuessLabels, text= guess[k], font=('Arial',17),bg=gylist[k],width=2,height=1)
                GuessLetter.grid(row=turn, column=k, sticky=tk.W+tk.E, padx=5, pady=5)
                GuessLabels.pack()
            #Checks if all letters are color Green (You got the word!):
            if check(gylist) is True and gylist[0]==Green:
                lose_Win_Length_label.config(text=f"You win! The word was {actualWord}")
                btnSubmit.config(state=DISABLED)
                if countdown_id:
                    root.after_cancel(countdown_id)
            # Covers the case where all letters are black or yellow on last turn(You lost!):
            elif turn>=6:
                lose_Win_Length_label.config(text=f"You Lose! The word was {actualWord}")
                btnSubmit.config(state=DISABLED)
                if countdown_id:
                    root.after_cancel(countdown_id)
        
            #Moves to next turn
            turn +=1
            
            #Clears the textbox 
            ClearText()
            

btnfrm2 = tk.Frame(root)
btnfrm2.columnconfigure(0, weight=1)
btnfrm2.columnconfigure(1, weight=1)
btnfrm2.columnconfigure(2, weight=1)


#Creates Submit button that activates "Actual_Game"
btnSubmit = tk.Button(btnfrm2, text ='Guess',font=('Arial',18),command=Actual_Game)
btnSubmit.grid()
btnfrm2.pack()

#Creates a Label to tell the user if they win or lose, and to give error if not 5 character guess:
lose_Win_Length_label = tk.Label(root, text= "", font=('Arial',24))
lose_Win_Length_label.pack()


# restart game function
def restart_game():
    global turn, actualWord, countdown_seconds, timer_label, countdown_id, gylist, GuessLabels

    # Reset game variables
    turn = 1
    actualWord = choose_random_word()
    countdown_seconds = 60
    gylist = ['#000000'] * 5

    # Reset elements
    textbox.delete("1.0", "end")
    lose_Win_Length_label.config(text="")
    btnSubmit.config(state=tk.NORMAL)

    # Reset button colors
    for btn in buttons:
        btn.config(fg='#000000')

    # Clear the previous five black squares
    if GuessLabels:
        # Destroy the existing GuessLabels frame
        GuessLabels.destroy()  
    GuessLabels = tk.Frame(root)

    # Create new labels and pack them
    for k in range(5):
        GuessLetter = tk.Label(GuessLabels, text="", font=('Arial', 17), bg='#FFFFFF', width=2, height=1)
        GuessLetter.grid(row=turn, column=k, sticky=tk.W + tk.E, padx=5, pady=5)

    # Cancel the ongoing countdown
    if countdown_id:
        root.after_cancel(countdown_id)
        countdown_id = None

    # Reset the timer label if it exists
    if timer_label:
        timer_label.config(text=f"Time Left: {countdown_seconds} seconds")

    # Pack the GuessLabels frame
    GuessLabels.pack()

# Create Restart button
Restart_Game = tk.Button(root, text="Restart Game", command=restart_game, font=('Arial', 18))
Restart_Game.pack()


root.mainloop()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
